<h1 align=center><font size = 6>Segmenting and Clustering Neighborhoods in Torono</font></h1>

<h1><font size = 5>Part 1 - Scraping data from Wikipedia</font></h1>

<h2><font size = 4>1.1 Importing the required libraries</font></h2>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# install and import the folium package
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

<h2><font size = 4>1.2 Scraping the required data from Wikipedia using BeautifulSoup</font></h2>

In [72]:
# Read the wikipedia HTML page related to the postal codes of Canada starting with M
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

# import the web scraping library and read the scrapped wikipedia page
from bs4 import BeautifulSoup
soup = BeautifulSoup(page,'lxml')

# The required table on the wikipedia page can be accessed via the tbody tag
My_table = soup.find('tbody')

<h2><font size = 4>1.3 Parsing and Cleaning the collected data </font></h2>

In [73]:
# Assign temporary variables to store the required data during scraping
PostalCode=[]
Borough=[]
Neighborhood=[]

rows = My_table.find_all('tr') # scan through the rows of the Wikipedia table, which are indicated via the tag 'tr'

for row in rows:
    cells = row.find_all('td') # scan through the columns of the Wikipadia table, which are indicated via the tag 'td'
    if len(cells) > 0:
        PostalC = cells[0].findAll(text=True)  # grab the postal code which is given in the first cell
        PostalC[0]=PostalC[0][0:3]             # select the first 3 characters from the string 
        Bor = cells[1].findAll(text=True)      # grab the Borough name which is given in the second cell
        Bor[0]=Bor[0][0:len(Bor[0])-1]         # select the appropriate characters from the string
        Nei = cells[2].findAll(text=True)      # grab the Neighborhoods which are given in the third cell
        Nei[0]=Nei[0][0:len(Nei[0])-1]         # select the appropriate characters from the string
        PostalCode.append(PostalC[0])          # Add the postal code
        Borough.append(Bor[0])                 # Add the Borough
        Neighborhood.append(Nei[0])            # Add the Neighborhoods

# store the scraped wikipedia data in a panda dataframe
df=pd.DataFrame(PostalCode,columns=['PostalCode'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood

#remove the not assigned Boroughs
indexNames = df[(df['Borough'] == 'Not assigned')].index
df.drop(indexNames , inplace=True)

#Fix the incorrect Neighborhood data by replacing the neighborhood with the borough
df['Neighborhood'].replace('Stn A PO Boxes',df['Borough'], inplace = True)
df['Neighborhood'].replace('Business reply mail Processing CentrE',df['Borough'], inplace = True)

#Correct the Neighborhood data to allign with the requested format
for Nei in df['Neighborhood']:
    NeiCorr = Nei.replace(' / ',', ')
    df['Neighborhood'].replace(Nei,NeiCorr, inplace = True)

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2><font size = 4>1.4 Number of rows of the dataframe containing the collected Wikipedia information </font></h2>

In [74]:
df.shape

(103, 3)

<h1><font size = 5>Part 2 - Adding latitude and longitude data </font></h1>

<h2><font size = 4>2.1 Getting the Neighborhood lattitude and longitude data </font></h2>

In [75]:
url= "http://cocl.us/Geospatial_data"
GeospacialData = pd.read_csv(url)
GeospacialData.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h2><font size = 4>2.2 Adding the geo data to our dataframe </font></h2>

In [76]:
latitude = []
longitude = []

for PostalC in df['PostalCode']:
    for id in range(0,len(GeospacialData)):
        if GeospacialData.loc[id,'Postal Code'] == PostalC:
            latitude.append(GeospacialData.loc[id,'Latitude'])
            longitude.append(GeospacialData.loc[id,'Longitude'])
        
df['Latitude']=latitude
df['Longitude']=longitude
df.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


<h1><font size = 5>Part 3 - Performing the analysis </font></h1>

<h2><font size = 4>3.1 Create a map of Toronto with the Boroughs superimposed on top </font></h2>

In [77]:
# create the Toronto map using folium
map_toronto = folium.Map(location=[43.651070,-79.347015], zoom_start=10)

# add Borough markets on top
for lat, lng, borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{},{}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

<h2><font size = 4>3.2 Narrowing our focus to 'downtown' Toronto </font></h2>

In [78]:
#remove the 4 Boroughs that are not part of 'downtown' Toronto
#these 4 Boroughs contain insufficient venue data to provide a clean K-means clustering
df = df[df.Borough != 'Etobicoke']
df = df[df.Borough != 'Scarborough']
df = df[df.Borough != 'North York']
df = df[df.Borough != 'Mississauga']

# create the Toronto map using folium
map_toronto = folium.Map(location=[43.651070,-79.347015], zoom_start=10)

# add the selected subset of Borough markers on top
for lat, lng, borough,neighborhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighborhood']):
    label = '{},{}'.format(Neighborhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

<h2><font size = 4>3.3 Exploring and segmenting the neighborhoods using Foursquare </font></h2>

<h3><font size = 3>3.3.1 Define Foursquare Credentials and Version </font></h3>

In [94]:
# The code was removed by Watson Studio for sharing.

<h3><font size = 3>3.3.2 Define the function to explore the neighborhoods via Foursquare </font></h3>

In [80]:
# We define a radius of 1000 i.s.o. 500 to secure sufficient venue information for clustering
# The logic behind this is that Toronto is less dense than New York city

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3><font size = 3>3.3.3 Collect the venue data and store in a new dataframe </font></h3>

In [81]:
df_venues = getNearbyVenues(names=df['Neighborhood'],
                            latitudes=df['Latitude'],
                            longitudes=df['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Woodbine Heights
St. James Town
Humewood-Cedarvale
The Beaches
Berczy Park
Caledonia-Fairbanks
Leaside
Central Bay Street
Christie
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Lawrence Park
Roselawn
Runnymede, The Junction North
Weston
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park


<h3><font size = 3>3.3.4 Inspect the information returned by Foursquare </font></h3>

In [82]:
print(df_venues.shape)
df_venues.groupby('Neighborhood').count()

print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

df_venues.head(5)

(3575, 7)
There are 285 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


<h3><font size = 3>3.3.5 Analyze Each Neighborhood </font></h3>

In [83]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot['Neighborhood'] = df_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]

df_onehot.head()

,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop

In [84]:
df_onehot.shape

(3575, 285)

<h3><font size = 3>3.3.6 Group the rows by neighborhood and the mean of the frequency of occurence of each category </font></h3>

In [85]:
df_grouped = df_onehot.groupby('Neighborhood').mean().reset_index()
df_grouped.head(5)

,Neighborhood,Zoo,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop

In [86]:
df_grouped.shape

(48, 285)

<h3><font size = 3>3.3.7 Overview of each neighborhood with the top 5 most common venues </font></h3>

In [87]:
num_top_venues = 5

for hood in df_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_grouped[df_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.13
1  Japanese Restaurant  0.05
2                 Café  0.05
3           Restaurant  0.05
4                Hotel  0.04
----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1                     Bar  0.05
2             Coffee Shop  0.05
3              Restaurant  0.05
4  Furniture / Home Store  0.04
----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst----
              venue  freq
0   Harbor / Marina  0.14
1       Coffee Shop  0.14
2             Track  0.07
3  Sculpture Garden  0.07
4           Airport  0.07
----Caledonia-Fairbanks----
           venue  freq
0       Bus Stop  0.08
1           Park  0.08
2       Pharmacy  0.08
3  Grocery Store  0.04
4       Bus Line  0.04
----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.09
1  Japanese Restaurant  0.04
2          Art Gallery  0.03
3                

<h3><font size = 3>3.3.8 Create a new dataframe containing the top 10 venues for each neighborhood </font></h3>

In [88]:
# function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(5)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Hotel,Park,Bakery,Gastropub,BBQ Joint
1,"Brockton, Parkdale Village, Exhibition Place",Café,Restaurant,Bar,Coffee Shop,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Supermarket,Park
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Harbor / Marina,Scenic Lookout,Sculpture Garden,Dog Run,Track,Park,Café,Dance Studio,Garden
3,Caledonia-Fairbanks,Pharmacy,Park,Bus Stop,Portuguese Restaurant,Bakery,Coffee Shop,Discount Store,Mexican Restaurant,Bus Line,Falafel Restaurant
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Art Gallery,Park,Italian Restaurant,Clothing Store,Ramen Restaurant,Mexican Restaurant,Gastropub,Electronics Store


<h3><font size = 3>3.3.9 Clustering the neighborhoods using K-means </font></h3>

In [90]:
# set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 1, 2, 2, 0, 1, 0, 0, 2, 2, 3, 0, 2, 1, 0, 2, 0, 0, 1, 1, 1, 1,
       4, 2, 1, 2, 2, 1, 1, 0, 2, 0, 2, 2, 1, 2, 0, 0, 1, 2, 1, 1, 1, 2,
       0, 1, 2, 2], dtype=int32)

<h3><font size = 3>3.3.10 Create a new dataframe including the cluster as well as the top 10 venues for each neighborhood </font></h3>

In [91]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Café,Pub,Theater,Diner,Park,Restaurant,Italian Restaurant,Breakfast Spot,Sushi Restaurant
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Park,Italian Restaurant,Pizza Place,Thai Restaurant,Gastropub,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Brewery,Fast Food Restaurant,Pizza Place,Gastropub,Bakery,Intersection,Rock Climbing Spot,Café,Athletics & Sports,Coffee Shop
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Ramen Restaurant,Italian Restaurant,Gastropub,Department Store,Sushi Restaurant,Hotel
21,M4C,East York,Woodbine Heights,43.695344,-79.318389,2,Coffee Shop,Pizza Place,Skating Rink,Park,Sandwich Place,Pharmacy,Restaurant,Spa,Convenience Store,Farmers Market


<h3><font size = 3>3.3.11 Visualizing the resulting custers on the Toronto map </font></h3>

In [92]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  
map_clusters

<h3><font size = 3>3.3.12 Examining the clusters </font></h3>

In [67]:
#Cluster 1 - Business district
# This cluster is characterized by the proxility of a lot of Hotels, Bars, Cafe's and coffe shops

df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,Coffee Shop,Park,Italian Restaurant,Pizza Place,Thai Restaurant,Gastropub,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint
13,Downtown Toronto,0,Coffee Shop,Clothing Store,Japanese Restaurant,Restaurant,Ramen Restaurant,Italian Restaurant,Gastropub,Department Store,Sushi Restaurant,Hotel
22,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Bakery,Italian Restaurant,Clothing Store,Seafood Restaurant,Cosmetics Shop,Breakfast Spot,Farmers Market
31,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Hotel,Park,Bakery,Gastropub,BBQ Joint
40,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Art Gallery,Park,Italian Restaurant,Clothing Store,Ramen Restaurant,Mexican Restaurant,Gastropub,Electronics Store
49,Downtown Toronto,0,Coffee Shop,Theater,Hotel,Restaurant,Café,Bakery,Clothing Store,Pizza Place,Asian Restaurant,Fast Food Restaurant
58,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Brewery,Gym,Japanese Restaurant,Park,Deli / Bodega,Theater
67,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Bar,Concert Hall,Italian Restaurant,Park,Beer Bar
76,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Gastropub,Beer Bar,Concert Hall,Deli / Bodega,Thai Restaurant
148,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Beer Bar,Bakery,Breakfast Spot,Gastropub,Seafood Restaurant,Italian Restaurant


In [68]:
#Cluster 2 - Residential area
# This cluster is characterized by the proxility of a lot of grocery stores, restaurants and the typical cafe's and coffee shops
# that are also present in cluster 1

df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East York,1,Brewery,Fast Food Restaurant,Pizza Place,Gastropub,Bakery,Intersection,Rock Climbing Spot,Café,Athletics & Sports,Coffee Shop
23,York,1,Pizza Place,Bagel Shop,Coffee Shop,Grocery Store,Convenience Store,Ice Cream Shop,Bank,Bakery,Frozen Yogurt Shop,Trail
30,East Toronto,1,Pub,Coffee Shop,Pizza Place,Beach,Japanese Restaurant,Breakfast Spot,Thai Restaurant,Café,Tea Room,Bar
41,Downtown Toronto,1,Korean Restaurant,Café,Coffee Shop,Grocery Store,Mexican Restaurant,Ice Cream Shop,Cocktail Bar,Karaoke Bar,Pizza Place,Diner
57,East York,1,Coffee Shop,Café,Park,Pizza Place,Greek Restaurant,Fast Food Restaurant,Bar,Brewery,Thai Restaurant,Ethiopian Restaurant
59,West Toronto,1,Café,Bar,Restaurant,Coffee Shop,Bakery,Pizza Place,Asian Restaurant,Cocktail Bar,Italian Restaurant,Bookstore
66,East Toronto,1,Greek Restaurant,Coffee Shop,Café,Pub,Fast Food Restaurant,Italian Restaurant,Pizza Place,Yoga Studio,Discount Store,Bookstore
68,West Toronto,1,Café,Restaurant,Bar,Coffee Shop,Bakery,Furniture / Home Store,Tibetan Restaurant,Gift Shop,Supermarket,Park
75,East Toronto,1,Indian Restaurant,Coffee Shop,Café,Grocery Store,Beach,Bakery,Burrito Place,Sandwich Place,Restaurant,Park
84,East Toronto,1,Coffee Shop,Bar,Café,Bakery,Brewery,Diner,Vietnamese Restaurant,American Restaurant,French Restaurant,Italian Restaurant


In [69]:
#Cluster 3 - Residential area with park facilities
# This cluster is characterized by the proxility of a lot of parks, on top a the amenities that are also present in cluster 1 and 2

df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,2,Coffee Shop,Café,Pub,Theater,Diner,Park,Restaurant,Italian Restaurant,Breakfast Spot,Sushi Restaurant
21,East York,2,Coffee Shop,Pizza Place,Skating Rink,Park,Sandwich Place,Pharmacy,Restaurant,Spa,Convenience Store,Farmers Market
32,York,2,Pharmacy,Park,Bus Stop,Portuguese Restaurant,Bakery,Coffee Shop,Discount Store,Mexican Restaurant,Bus Line,Falafel Restaurant
39,East York,2,Sporting Goods Shop,Coffee Shop,Furniture / Home Store,Grocery Store,Electronics Store,Burger Joint,Sports Bar,Brewery,Bank,Sandwich Place
48,East York,2,Coffee Shop,Grocery Store,Indian Restaurant,Turkish Restaurant,Brewery,Burger Joint,Shopping Mall,Bank,Pizza Place,Supermarket
50,West Toronto,2,Café,Coffee Shop,Park,Bar,Sushi Restaurant,Portuguese Restaurant,Brewery,Bakery,Italian Restaurant,Pharmacy
94,Central Toronto,2,Italian Restaurant,Café,Sushi Restaurant,Pharmacy,Bank,Coffee Shop,Clothing Store,Skating Rink,Dry Cleaner,Japanese Restaurant
95,York,2,Pizza Place,Coffee Shop,Park,Brewery,Bus Line,Pharmacy,Gas Station,Beer Store,Sandwich Place,Discount Store
98,York,2,Coffee Shop,Train Station,Pizza Place,Park,Soccer Field,Skating Rink,Café,Sandwich Place,Diner,Fried Chicken Joint
102,Central Toronto,2,Coffee Shop,Italian Restaurant,Dessert Shop,Café,Pizza Place,Pharmacy,Gym,Sushi Restaurant,Yoga Studio,Pub


In [70]:
#Cluster 4 - Area with limited residential facilities but close to a park and playground
# This cluster is characterized by the proxility of a large park and a playground

df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
86,York,3,Furniture / Home Store,Grocery Store,Fast Food Restaurant,Sandwich Place,Italian Restaurant,Park,Gas Station,Restaurant,Playground,Wine Shop


In [71]:
#Cluster 5 - Area characterized by parks and Colleges

df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[1] + list(range(5, df_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
93,Central Toronto,4,Café,Gym / Fitness Center,Bookstore,Park,College Quad,College Gym,Coffee Shop,Trail,Yoga Studio,Doner Restaurant
